In [1]:
import pandas as pd
import numpy as np
import json

In [ ]:
DATA_DIR = "./../data/"
OUTPUT_DIR = "./../output/datasets"

In [ ]:
df = []
with open(OUTPUT_DIR + "defSelected.jsonl", 'r') as f:
    for line in f:
        df.append(json.loads(line))

df = pd.DataFrame(df)

In [8]:
icd10_df = pd.read_feather(DATA_DIR + "mimiciv_icd10.feather")

In [7]:
df

,note_id,selected
0,10052875-DS-13,"[Thoracic aortic ectasia, Anal fistula, Anemia..."
1,10788434-DS-13,[Calculus of gallbladder without cholecystitis...
2,13211200-DS-22,"[Peritoneal abscess, Hyperlipidemia unspecifie..."
3,11819384-DS-21,"[Chronic diastolic (congestive) heart failure,..."
4,11992544-DS-10,[Dementia in other diseases classified elsewhe...
...,...,...
95,13501710-DS-6,"[Solitary pulmonary nodule, Gastro-esophageal ..."
96,15776313-DS-14,"[Unspecified asthma uncomplicated, Chronic kid..."
97,13031383-DS-20,"[Chronic diastolic (congestive) heart failure,..."
98,14113167-DS-2,[Gastro-esophageal reflux disease without esop...


In [11]:
data = df.merge(icd10_df, how="inner", on="note_id")[["note_id", "icd10_diag_titles", "selected"]]

In [19]:
!pip install evaluate

     ---------------------------------------- 84.0/84.0 kB 4.9 MB/s eta 0:00:00
     ---------------------------------------- 78.6/78.6 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.3
    Uninstalling tqdm-4.62.3:
      Successfully uninstalled tqdm-4.62.3


In [36]:
import evaluate
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score

In [39]:
# Convertiamo le stringhe in etichette numeriche
# 1. Troviamo tutte le etichette uniche
all_labels = set(label for sublist in data['icd10_diag_titles'].tolist() + data['selected'].tolist() for label in sublist)

# 2. Usare un binarizzatore multi-etichetta di scikit-learn per trasformare le stringhe in valori numerici
mlb = MultiLabelBinarizer(classes=list(all_labels))
true_labels_bin = mlb.fit_transform(data['icd10_diag_titles'])
predicted_labels_bin = mlb.transform(data['selected'])

# Inizializza la metrica F1 con Hugging Face Evaluate
f1_metric = evaluate.load("f1")

# Calcola la F1 macro e micro con sklearn
f1_macro = f1_score(true_labels_bin, predicted_labels_bin, average="macro", zero_division=0)
f1_micro = f1_score(true_labels_bin, predicted_labels_bin, average="micro", zero_division=0)
precision_macro = precision_score(true_labels_bin, predicted_labels_bin, average="macro", zero_division=0)
recall_macro = recall_score(true_labels_bin, predicted_labels_bin, average="macro", zero_division=0)
precision_micro = precision_score(true_labels_bin, predicted_labels_bin, average="micro", zero_division=0)
recall_micro = recall_score(true_labels_bin, predicted_labels_bin, average="micro", zero_division=0)

print("F1 Macro:", f1_macro)
print("Precision Macro:", precision_macro)
print("Recall Macro:", recall_macro)
print("F1 Micro:", f1_micro)
print("Precision Micro:", precision_micro)
print("Recall Micro:", recall_micro)


F1 Macro: 0.10847914912796289
Precision Macro: 0.10263686241351601
Recall Macro: 0.13956538127126364
F1 Micro: 0.212998361551065
Precision Micro: 0.17873510540788268
Recall Micro: 0.2635135135135135
